In [4]:
#pip install gym

In [3]:
#load packages
import gym
import numpy as np
from gym import spaces
from scipy.stats import norm

In [5]:
class invasive_IPM(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        params={"growth_k": 0.43, "growth_xinf": 109, "growth_sd": 2.5, "nmortality": 0.03,
               "trapm_sigma": 0.15, "trapm_xmax": 44, "trapm_pmax": 0.0005, "trapf_pmax": 0.0008,
               "trapf_k": 0.5, "trapf_midpoint": 45, "init_mean_recruit": 15, "init_sd_recruit": 1.5,
               "init_mean_adult": 65, "init_sd_adult": 8, "init_n_recruit": 1000, "init_n_adult": 1000,
               "w_mort_scale": 5, "K": 30000, "imm": 3000, "r": 1.2, "area": 4000,"loss_a": 0.265,
               "loss_b": 2.80, "loss_c": 2.99, "minsize": 5, "maxsize": 110, "nsize": 21, "delta_t": 1/12},
        Tmax=100,
        file=None,
    ):
        
        # parameters
        self.growth_k = params["growth_k"]
        self.growth_xinf = params["growth_xinf"]
        self.growth_sd = params["growth_sd"]
        self.nmortality = params["nmortality"]
        self.trapm_sigma = params["trapm_sigma"]
        self.trapm_xmax = params["trapm_xmax"]
        self.trapm_pmax = params["trapm_pmax"]
        self.trapf_pmax = params["trapf_pmax"]
        self.trapf_k = params["trapf_k"]
        self.trapf_midpoint = params["trapf_midpoint"]
        self.init_mean_recruit = params["init_mean_recruit"]
        self.init_sd_recruit = params["init_sd_recruit"]
        self.init_mean_adult = params["init_mean_adult"]
        self.init_sd_adult = params["init_sd_adult"]
        self.init_n_recruit = params["init_n_recruit"]
        self.init_n_adult = params["init_n_adult"]
        self.w_mort_scale = params["w_mort_scale"]
        self.K = params["K"]
        self.imm = params["imm"]
        self.r = params["r"]
        self.area = params["area"]
        self.loss_a = params["loss_a"]
        self.loss_b = params["loss_b"]
        self.loss_c = params["loss_c"]
        self.minsize = params["minsize"]
        self.maxsize = params["maxsize"]
        self.nsize = params["nsize"]
        self.delta_t = params["delta_t"]
        self.params = params

        # Preserve these for reset
        self.population = init_state(self)
        self.observations = np.array([0,0,0,0,0,0,0,0,0], dtype=np.float32)
        self.reward = 0
        self.years_passed = 0
        self.Tmax = Tmax
                
        # Initial state
        self.state = init_state(self)

        # Action space
        self.action_space = spaces.Box(
            np.array([0], dtype=np.float32),
            np.array([1000], dtype=np.float32),
            dtype=np.float32,
        )
        
        # Observation space
        self.observation_space = spaces.Box(
            np.array([0,0,0,0,0,0,0,0,0], dtype=np.float32),
            np.array([2000,2000,2000,2000,2000,2000,2000,2000,2000], dtype=np.float32),
            dtype=np.float32,
        )
        
    def step(self,action):
        #size selective harvest rate, given action
        harvest_rate = 1-np.exp(-(size_sel_norm(self)*action/2 + size_sel_log(self)*action/2))

        #add pop at t=1
        size_freq = np.zeros(shape=(self.nsize,9),dtype='object')
        size_freq[:,0] = self.state

        #create array to store # removed
        removed = np.empty(shape=(self.nsize,9),dtype='object')

        #loop through intra-annual change (9 months)
        for i in range(9):
            #apply monthly harvest rate
            removed[:,i] = size_freq[:,i]*harvest_rate
        
        for j in range(8):
            #project to next month
            size_freq[:,j+1] = g_m_kernel(self)@size_freq[:,j] - removed[:,j]

        for i in range(9):
            #record the catch/effort in the observation space
            #### maybe remove effort
            self.observations[i] = np.sum(removed[:,i])/action

        #calculate new adult population after overwinter mortality
        new_adults = size_freq[:,8]*np.exp(-w_mortality(self))

        #simulate new recruits
        local_recruits = np.random.normal(dd_growth(self),dd_growth(self)/4)
        nonlocal_recruits = np.random.normal(self.imm,self.imm/4)*(1-np.sum(self.state)/self.K)
        recruit_total = local_recruits + nonlocal_recruits

        #get sizes of recruits
        recruit_sizes = (norm.cdf(boundary(self)[1:(self.nsize+1)],self.init_mean_recruit,self.init_sd_recruit)-\
         norm.cdf(boundary(self)[0:self.nsize],self.init_mean_recruit,self.init_sd_recruit))*recruit_total

        #store new population size
        self.state = recruit_sizes + new_adults

        #calculate reward
        self.reward = reward_func(self)
        self.years_passed += 1

        done = bool(self.years_passed > self.Tmax)

        if np.sum(self.state) <= 0.0:
            done = True

        return self.observations, self.reward, done, done, {}
        
    def reset(self):
        self.state = init_state(self)
        self.years_passed = 0

        # for tracking only
        self.reward = 0

        self.observations = np.array([0,0,0,0,0,0,0,0,0], dtype=np.float32)

        return self.observations

In [58]:
self = invasive_IPM()

In [77]:
action = 25
self.reset()
-self.loss_a/(1+np.exp(self.loss_b*(np.sum(self.state)/4000-self.loss_c)))

-0.2647517223286356

In [6]:
#set up boundary points of IPM mesh
def boundary(self):
    boundary = self.minsize+np.arange(0,(self.nsize+1),1)*(self.maxsize-self.minsize)/self.nsize
    return boundary

In [7]:
#set up mid points of IPM mesh
def midpoints(self):
    midpoints = 0.5*(boundary(self)[0:self.nsize]+boundary(self)[1:(self.nsize+1)])
    return midpoints

In [8]:
#function for initial state
def init_state(self):
    init_pop = (norm.cdf(boundary(self)[1:(self.nsize+1)],self.init_mean_adult,self.init_sd_adult)-\
     norm.cdf(boundary(self)[0:self.nsize],self.init_mean_adult,self.init_sd_adult))*self.init_n_adult+\
    (norm.cdf(boundary(self)[1:(self.nsize+1)],self.init_mean_recruit,self.init_sd_recruit)-\
     norm.cdf(boundary(self)[0:self.nsize],self.init_mean_recruit,self.init_sd_recruit))*self.init_n_recruit
    return init_pop

In [9]:
#function for logistic size selectivity curve
def size_sel_log(self):
    size_sel = self.trapf_pmax/(1+np.exp(-self.trapf_k*(midpoints(self)-self.trapf_midpoint)))
    return size_sel

In [10]:
#function for gaussian size selectivity curve
def size_sel_norm(self):
    size_sel = self.trapm_pmax*np.exp(-(midpoints(self)-self.trapm_xmax)**2/2*self.trapm_sigma**2)
    return size_sel

In [11]:
#function for growth/mortality kernel
def g_m_kernel(self):
    array = np.empty(shape=(self.nsize,self.nsize),dtype='object')
    for i in range(self.nsize):
        mean = (self.growth_xinf-midpoints(self)[i])*(1-np.exp(-self.growth_k*self.delta_t)) + midpoints(self)[i]
        array[:,i] = (norm.cdf(boundary(self)[1:(self.nsize+1)],mean,self.growth_sd)-\
                      norm.cdf(boundary(self)[0:self.nsize],mean,self.growth_sd))*np.exp(-self.nmortality)
    return array

In [12]:
#function for overwinter mortality
def w_mortality(self):
    wmort = self.w_mort_scale/midpoints(self)
    return wmort

In [13]:
#function for density dependent growth
def dd_growth(self):
    dd_recruits = np.sum(self.state)*self.r*(1-np.sum(self.state)/self.K)
    return dd_recruits

In [14]:
#function for reward
def reward_func(self):
    reward = -self.loss_a/(1+np.exp(self.loss_b*(np.sum(self.state)/self.area-self.loss_c)))
    return reward

In [16]:
env=invasive_IPM()   

In [41]:
episode_reward = 0
env.reset()
env.reward

0

In [17]:
observation,reward,terminated,done,info=env.step(25)
reward

-0.25783387283049014

In [89]:
env.reward

0

In [96]:
episode_reward = 0.0
for t in range(30):
    observation,reward,terminated,done,info=env.step(25)
    episode_reward += reward
    #print(observation)
    print(episode_reward)

-1.3483094020399223e-05
-0.0004210189285177259
-0.0004395885727792677
-0.0004924467100960526
-0.0010904159524171941
-0.001226258918644944
-0.0016484403258451574
-0.0016534365109421199
-0.0018916868669725394
-0.002243381523420129
-0.00234601902388302
-0.0023627863950251493
-0.002423250116212694
-0.002436043586220065
-0.002625696352814231
-0.002642750152529128
-0.0028112526685008703
-0.002932984285610109
-0.0029515669760694675
-0.003089726980346676
-0.0031651936311454616
-0.003310453202407089
-0.0035916239358029993
-0.004356088526554937
-0.004410564121675883
-0.004486155436431395
-0.004515783149220741
-0.004597079755695389
-0.004630113641069404
-0.00473617458325576
